# Project Template

Copy this repository to get started with the project

## Setup

Modify the following 2 cells to import libraries and helper files that you might need for your project

In [1]:
# Some possible helper files from class

!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/audio_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/image_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/text_utils.py

In [2]:
# Some possible libraries.
# This isn't complete.

import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image as PImage

from os import listdir, path

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC

from data_utils import object_from_json_url, classification_error, display_confusion_matrix
from image_utils import get_pixels, make_image
from text_utils import get_top_words

In [3]:
df = pd.read_csv("FINAL data_311_complaints_2014 only_ CSV.csv")
df.head()

,Unique Key,Month,Date,Year,Time,Complaint Type,Descriptor,Location Type,Incident Zip,Community Board,Latitude,Longitude,Location
0,29610024,12,31,2014,23:54,Noise - Residential,Loud Music/Party,Residential Building/House,10030,10 MANHATTAN,40.814236,-73.945203,"(40.81423607337286, -73.94520343689118)"
1,29612307,12,31,2014,23:50,Noise - Residential,Loud Music/Party,Residential Building/House,10030,10 MANHATTAN,40.821400,-73.939261,"(40.82139955933966, -73.93926132645576)"
2,29611549,12,31,2014,23:50,Noise - Commercial,Loud Music/Party,Store/Commercial,10026,10 MANHATTAN,40.802798,-73.956540,"(40.80279815154175, -73.95654015432075)"
3,29607558,12,31,2014,23:47,Noise - Residential,Loud Music/Party,Residential Building/House,10031,09 MANHATTAN,40.819895,-73.955236,"(40.81989452751751, -73.95523555304263)"
4,29609974,12,31,2014,23:40,Noise - Residential,Loud Music/Party,Residential Building/House,10030,10 MANHATTAN,40.821400,-73.939261,"(40.82139955933966, -73.93926132645576)"


## Milestone 01

### Short Description... maybe... ?

In [ ]:
# TODO: Code for project